In [5]:
!nvidia-smi


Tue Nov 21 20:35:56 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    26W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

!pip install --upgrade accelerate -q
!pip uninstall -y transformers accelerate -q
!pip install transformers accelerate -q

In [7]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import DataCollatorForSeq2Seq
from transformers import TrainingArguments, Trainer
from transformers import pipeline
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from datasets import DatasetDict, Dataset, load_dataset

from datasets import load_dataset, load_from_disk, load_metric

import pandas as pd

import nltk
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

from tqdm import tqdm
import torch

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:

# torch.cuda.is_available() function to check if a GPU is available for use else CPU will be used

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [9]:

# Pre-trained model checkpoint for the PEGASUS model

model_ckpt = "google/pegasus-cnn_dailymail"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)
# .to(device)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
df = pd.read_csv('/content/drive/Shareddrives/298_B_final_Sem/Pegasus/summarization_cleaned_file_Himanshee(9-14).csv')

In [12]:
limit=40000
if limit!=-1:
  df=df.head(limit)

In [13]:
# Split the data into 70% train and 30% rest (which will be used for test and val)
train_df, rest_df = train_test_split(df, test_size=0.3, random_state=42)

# Split the rest into 50% test and 50% validation
test_df, val_df = train_test_split(rest_df, test_size=0.5, random_state=42)

In [14]:
train = Dataset.from_pandas(train_df)
val = Dataset.from_pandas(val_df)
test = Dataset.from_pandas(test_df)

full_dataset = DatasetDict({'train': train, 'val': val, 'test': test})

In [15]:
print(full_dataset)

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', '__index_level_0__'],
        num_rows: 7000
    })
    val: Dataset({
        features: ['article', 'highlights', '__index_level_0__'],
        num_rows: 1500
    })
    test: Dataset({
        features: ['article', 'highlights', '__index_level_0__'],
        num_rows: 1500
    })
})


In [16]:
!pip install rouge


In [17]:
def generate_summary(batch):
    # Ensure padding and truncation
    inputs = tokenizer(batch["article"], return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
    outputs = model_pegasus.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=120,
        do_sample=False,
        #top_p=0.9
    )
    # Decode all generated summaries
    summaries = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    return {"summary": summaries}

In [18]:
from rouge import Rouge
# print(type(test[0].items))
summary_dataset = test.map(generate_summary, batched=True, batch_size=10)

# # Extract generated and reference summaries
generated_summaries = summary_dataset["summary"]
reference_summaries = [item["highlights"] for item in test]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [19]:
from datasets import load_dataset, load_metric

metric = load_metric("rouge")

<ipython-input-19-faaac16489c9>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


In [20]:
# Now compute ROUGE scores
scores = metric.compute(predictions=generated_summaries, references=reference_summaries)

# Output the scores
print(scores)
rouge_l_score = scores["rougeL"].mid.fmeasure
rouge_n1_score = scores["rouge1"].mid.fmeasure
rouge_n2_score = scores["rouge2"].mid.fmeasure

{'rouge1': AggregateScore(low=Score(precision=0.2920922406792105, recall=0.4278392530611462, fmeasure=0.3349107997977474), mid=Score(precision=0.29845955673142033, recall=0.4350611854819359, fmeasure=0.34070008432667886), high=Score(precision=0.304665582614323, recall=0.44243327193521986, fmeasure=0.34630549953009115)), 'rouge2': AggregateScore(low=Score(precision=0.11782510840020217, recall=0.17145731246641152, fmeasure=0.13458994768366359), mid=Score(precision=0.1227111302834354, recall=0.17773748944609563, fmeasure=0.13961739855558225), high=Score(precision=0.12771464379628297, recall=0.18464550491909687, fmeasure=0.14472362087256396)), 'rougeL': AggregateScore(low=Score(precision=0.20242153368369087, recall=0.29759940625226233, fmeasure=0.23245233901012743), mid=Score(precision=0.20730203416175003, recall=0.30392673997324726, fmeasure=0.23716435193197602), high=Score(precision=0.21241592181981378, recall=0.3099518200584227, fmeasure=0.2419713536551525)), 'rougeLsum': AggregateScore

In [21]:
print(f"RougeL: {rouge_l_score:.4f} - RougeN1: {rouge_n1_score:.4f} - RougeN2: {rouge_n2_score:.4f}")

RougeL: 0.2372 - RougeN1: 0.3407 - RougeN2: 0.1396
